# 0a. Creating a separate object detector just for the license plates

In [ ]:
# importing the training data
!pip install roboflow
from roboflow import Roboflow
rf = Roboflow(api_key="xyz")
project = rf.workspace("mochoye").project("license-plate-detector-ogxxg")
dataset = project.version(2).download("yolov8")

In [ ]:
from ultralytics import YOLO

In [ ]:
lp_model = YOLO('yolov8n.pt')  # load a pretrained model (recommended for training)
results = lp_model.train(data='/Users/fedoradushev/Documents/GitHub/Computer-Vision/yolo/Projects/license_plate_detector/License-Plate-Detector-2/data.yaml', 
                         epochs=100, imgsz=640, device="mps")   # train the model

# 0b. Link licence plate to a vehicle

`abewley/sort`: can assign an id to the tracked object based on the information of the bbox of two frames

```
from sort.sort import *

mot_tracker = Sort()

track_ids = mot_tracker.update(np.asarray(detections))   # adds an additional column to 'detections_' for vehicle-id

```

detections: x1, y1, x2, y2, score, class_id


In [2]:
def get_car(license_plate, vehicles):
    
    x1, y1, x2, y2, score, class_id = license_plate

    foundIt = False
    
    for j in range(len(vehicles)):
        x1_carj, y1_carj, x2_carj, y2_carj, carj_score = vehicles[j]

        if x1 > x1_carj and y1 > y1_carj and x2 < x2_carj and y2 < y2_carj:
            car_indx = j
            foundIt = True
            break

    if foundIt:
        return vehicles[car_indx]

    return -1, -1, -1, -1, -1

# 0c. Read the license

In [3]:
import easyocr

reader = easyocr.Reader(['en'], gpu=True)

def read_lp(license_plate_crop):

    detections = reader.readtext(license_plate_crop)

    for detection in detections:
        bbox, text, score = detection

        text = text.upper()
        text = text.replace(' ', '')

        #text = format_text(text)
        return text

# 01. Automatic licence plate recognition

In [ ]:
 # enable contiuous tracking
!git clone https://github.com/abewley/sort.git

In [4]:
from ultralytics import YOLO
import cv2

lp_model = YOLO('/Users/fedoradushev/Documents/GitHub/Computer-Vision/yolo/Projects/license_plate_detector/runs/detect/train/weights/last.pt')
coco_model = YOLO('yolov8n.pt')

included classes of the [coco-model](https://tech.amikelive.com/node-718/what-object-categories-labels-are-in-coco-dataset/)
| ID | OBJECT (PAPER) | OBJECT (2014 REL.) | OBJECT (2017 REL.) | SUPER CATEGORY |
|----|----------------|--------------------|--------------------|----------------|
| 1  | person         | person             | person             | person         |
| 2  | bicycle        | bicycle            | bicycle            | vehicle        |
| 3  | car            | car                | car                | vehicle        |
| 4  | motorcycle     | motorcycle         | motorcycle         | vehicle        |
| 5  | airplane       | airplane           | airplane           | vehicle        |
| 6  | bus            | bus                | bus                | vehicle        |
| 7  | train          | train              | train              | vehicle        |
| 8  | truck          | truck              | truck              | vehicle        |
| 9  | boat           | boat               | boat               | vehicle        |
| 10 | traffic light  | traffic light      | traffic light      | outdoor        |

In [5]:
# selecting the relevant classes for the tracking of vehicles

vehicles = [2,   # car
            3,   # motorcycle
            5,   # bus
            7]   # truck

In [6]:
def draw_bbox_lp(frame, x1, y1, x2, y2):
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 5)

# corresponding bbox of car to previous lp
def draw_bbox_car(frame, x1, y1, x2, y2):
    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 0, 255), 5)
    
def display_lp(frame, x1, y1, lp_frame_crop):
    lp_gray = cv2.cvtColor(lp_frame_crop, cv2.COLOR_BGR2GRAY)      
    _, lp_thresh = cv2.threshold(lp_gray, 64, 255, cv2.THRESH_BINARY_INV)   # makes it easier for the ocr software to process

    # ocr on license plate
    lp_txt = read_lp(lp_thresh)

    cv2.putText(frame, lp_txt, (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 0), 2)

In [7]:
# DTECTION OF ALL VEHICLES IN THE CURRENT FRAME
def detect_all_vehicles_in_frame(frame, coco_model, vehicles):
    detections = coco_model(frame)[0]
    detections_ = list()              # storage for the vehicle bBoxes & their probability → we don't care about what type of vehicle that is

    for detection in detections.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = detection
        if int(class_id) in vehicles:
            detections_.append([x1, y1, x2, y2, score])
    return detections_

In [8]:
# DETECTION OF ALL LICENSE PLATES IN THE CURRENT FRAME
def detect_all_license_plates_in_frame(frame, lp_model, detection_):
    license_plates = lp_model(frame)[0]
    for license_plate in license_plates.boxes.data.tolist():
        x1, y1, x2, y2, score, class_id = license_plate

        draw_bbox_lp(frame, x1, y1, x2, y2)
        
        # assign license plate to car
        x1_carj, y1_carj, x2_carj, y2_carj, carj_score = get_car(license_plate, detections_)
        
        if carj_score != -1:
            draw_bbox_car(frame, x1_carj, y1_carj, x2_carj, y2_carj)

        if carj_score != -1:
            # crop the licence plate
            lp_frame_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
            display_lp(frame, x1_carj, y1_carj, lp_frame_crop)

the utilized video can be downloaded [here](https://drive.google.com/file/d/12sBfgLICdQEnDSOkVFZiJuUE6d3BeanT/view)

In [ ]:
video = cv2.VideoCapture('./highway_full.mp4')

frame_nr = -1
ret = True

while ret:
    frame_nr += 1
    ret, frame = video.read()
    
    
    if ret:
        # ALL CARS
        detections_ = detect_all_vehicles_in_frame(frame, coco_model, vehicles)
        # ALL LP
        license_plates = lp_model(frame)[0]
            # ITERATING OVER ALL LP
        for license_plate in license_plates.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = license_plate

            draw_bbox_lp(frame, x1, y1, x2, y2)

            # assign license plate to car
            x1_carj, y1_carj, x2_carj, y2_carj, carj_score = get_car(license_plate, detections_)

            if carj_score != -1:
                draw_bbox_car(frame, x1_carj, y1_carj, x2_carj, y2_carj)

            if carj_score != -1:
                # crop the licence plate
                lp_frame_crop = frame[int(y1):int(y2), int(x1): int(x2), :]
                display_lp(frame, x1_carj, y1_carj, lp_frame_crop)
        
        
        cv2.imshow('frame', frame)
        cv2.waitKey(1)
        
        
video.release()           
cv2.destroyAllWindows()


0: 384x640 22 cars, 1 bus, 2 trucks, 59.1ms
Speed: 3.7ms preprocess, 59.1ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 LicensePlate, 53.6ms
Speed: 2.6ms preprocess, 53.6ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 147.7ms
Speed: 2.3ms preprocess, 147.7ms inference, 0.7ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 LicensePlate, 56.5ms
Speed: 1.9ms preprocess, 56.5ms inference, 0.4ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 22 cars, 1 bus, 2 trucks, 61.2ms
Speed: 2.5ms preprocess, 61.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 57.0ms
Speed: 2.0ms preprocess, 57.0ms inference, 0.2ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 21 cars, 1 bus, 2 trucks, 93.5ms
Speed: 10.1ms preprocess, 93.5ms inference, 1.0ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 LicensePlate, 57.2